In [386]:
%matplotlib inline

In [387]:
import matplotlib.pyplot as plt
import numpy as np
import time
import math
from collections import Counter

from tiles import *

In [387]:
plt.xkcd()  # Yes...

tmax = 4000

t = tmax*np.linspace(0, 1, tmax, endpoint=False)

targetElbow = np.zeros(tmax)

steps = tmax / 4
for i in range(0, tmax, steps):
    
    # Define the target angles 
    targetElbow[i:i+(steps/4)] = 0
    targetElbow[i+(steps/4):i+(steps/2)] = math.pi/2
    targetElbow[i+(steps/2):i+(3*steps/4)] = math.pi
    targetElbow[i+(3*steps/4):i+steps] = math.pi/2

# Two subplots, unpack the axes array immediately
plt.figure(figsize=(40,40))
f, (ax1, ax2) = plt.subplots(2, 1, sharey=False)

ax1.plot(t, targetElbow)
ax1.set_ylim([-math.pi/2, 4])
ax1.set_title('Control Signal')

delay = 10

noise = np.random.normal(0,0.1,tmax)
simEMG = [0] * (tmax+delay)

for i in range(1,tmax):
    if targetElbow[i] == math.pi/2:
        simEMG[i + delay] = -1
    elif targetElbow[i] < math.pi/2:
        simEMG[i + delay] = 0
    elif targetElbow[i] > math.pi/2:
        simEMG[i + delay] = 1
        
simEMGdiff = simEMG[0:tmax]+noise

ax2.plot(t, simEMGdiff)
ax2.set_ylim(-1.5, 1.5)
ax2.set_title('Simulated Noisy EMG Signal')

KeyboardInterrupt: 

In [ ]:
# Implement a learning algorithm to try to fit the signal

# store the variables for plitting after learning
jointAngle = np.zeros(tmax)

# Initialize the state array for the trajectory
# Possible feedback from the simulated arm includes the angle
# (in radians) and angular velocity (in radians per second) of
# each joint, and the Cartesian position of the end effector.

# The continuous state space is composed of joint angles 
# and differential EMG signals, for simplicity we will use 
# single joint angle and a single emg differetial signal
s = np.array([0,0])

# lambdw = 0.3
# lambdv = 0.7

x = 0

# Initialize the learning parameters
gamma = 0.99
lambd = 0.3
alphaV = 0.1/m
alphaW = 0.01/m

# Standard deviation should cover the possible action set
maxAngVelInt = 1023
stdC = maxAngVelInt

r = 0

delta = np.zeros(tmax)

agentMean = np.zeros(tmax)
agentStd = np.zeros(tmax)

# this defines the resolution of the tiling
numTilings = 25
resolutions = np.array([8])
# resolutions = np.array([5,8,12,20])
numFeatures = len(s)

# Actual length is the concatenation at different resolutions
# and the baseline feature
featVecLength = sum(np.power(resolutions,numFeatures)*numTilings)+1

# Initialize the weight vectors to zero 
# the should be as long as the feature vector x
elV = np.zeros(featVecLength)
elU = np.zeros(featVecLength)
elS = np.zeros(featVecLength)
v = np.zeros(featVecLength)
wU = np.zeros(featVecLength)
wS = np.zeros(featVecLength)

# This function converts the angular velocity from an integer in the range [-1023,1023]
# to an angular velocity in radians per second.
# The no-load speed of the motor is ~60RPM, which is around 6 radians per second
# rad/s = rot(RPM) 2*pi/60
# This should give a good simulation of the true robotic kinematics
def conAngIntToangVel(angInt):
    motorVel = 2
    angVel = motorVel * angInt / maxAngVelInt
    return angVel

def featurize(s):
    
    # TODO: all components in s were normalized to the range [0, 1]
    # according to their minimum and maximum possible values
    # (((OldValue - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
    newAngle = (((s[0] - 0) * (1 - 0)) / (math.pi - 0)) + 0
    newEMG = (((s[1] - (-1.5)) * (1 - 0)) / (1.5 - (-1.5))) + 0

    x = np.zeros(featVecLength)
    
    # Single active baseline unit
    x[0] = 1 
    
    # For each resolution the features are needed 
    for res in np.nditer(resolutions):
#         floatArray = np.array([newAngle*res,newEMG*res])
        floatArray = np.array([newAngle*res])
 
        # Get the indexes of the tiles
        tilesOut = tiles(numTilings, res**2, floatArray) 
        
        # For each tile index flip a feature in the full feature vector
        for i in range(len(tilesOut)):
            x[tilesOut[i] + res**2 * (i)] = 1
    
    # x is the active tiles in this featurization
    
    # This featurization should return a single binary feature vector
    # with exactly m features in x(s) active at any time
    
    # Binary feature vector has a length of 4636426 or
    # sum(np.power(np.array([5,8,12,20]),4)*25)+1
    
    return x

def getReward(newAngle,timeStep):
    # Define the reward function of the system
    # A positive reward of rt = 1.0 was
    # delivered when θw and θe were both within 0.1 radians of
    # their target angles. A reward of rt = −0.5 was delivered
    # in all other cases, in essence penalizing the learning system
    # when the arm’s posture differed from the target posture.
    targetAngle = targetElbow[timeStep]
    
    absAngleError = np.abs(newAngle-targetAngle)
    
    if absAngleError < 0.1:
        r = 0.1
    else:
        r = -0.5

    return r

def perform(vel,s,timeStep):
    
    # take the action and observe the new state and the reward 
    # new state is defined by the new joint angle
    # which is defined by the old joint angle and the new angular velocity
    # which is applied for that time step and the
    # emg signal at that time index 
    
    # Calculate the new angular state of the joint
    # old angle + angular velocity * time (in this case time = 5ms, the period of action selection)
    # this limits the amount of motion of the joint possible in each action selection
    
    # newAngle = old angle + (angular velocity * time)
    newAngle = s[0] + (vel * 0.05)
    
    # Angle of the joint is limited
    if newAngle < 0:
        newAngle = 0
    elif newAngle > math.pi:
        newAngle = math.pi
    
    # Define the new state space with the new angle and the emg information
    s = np.array([newAngle,simEMGdiff[timeStep]])
    
    # Get the reward for the new angle
    r = getReward(newAngle,timeStep)
    
    return r,s

# process the training samples that are given
for i in range(tmax):
    
    # Store joint angles for plotting
    jointAngle[i] = s[0]
   
    # Featurize the state
    x = featurize(s)
    
    # Calculate the mean of the action selection
    agentMean[i] = np.dot(wU,x)
    
#     print np.exp(np.dot(wS,x)+np.log(stdC))
    agentStd[i] = max(1,np.exp(np.dot(wS,x) + np.log(stdC)))
    
    # get the action from the normal distribution
    # In cases where more than one action (a is required, 
    # one set of actor parameter vectors (wµ, wσ, eµ, eσ)
    # is maintained for each action (e.g., Tamei and Shibata [17]).

    # angular velocity commands are sent to joints (simulated servos) 
    # as integers in the range [−maxAngVelInt, maxAngVelInt] 
    a = round(np.random.normal(agentMean[i],agentStd[i]))    
    if a < -maxAngVelInt:
        a = -maxAngVelInt
    elif a > maxAngVelInt:
        a = maxAngVelInt
 
    # take action a and observe the reward, r, and the new state, s
    angVel = conAngIntToangVel(a)
    r,s = perform(angVel,s,i)              
    newX = featurize(s)

    # Calculate the TD Error based on the old state and the new state
    delta[i] = r + (gamma * np.dot(v,newX)) - np.dot(v,x)
    
    # Critic's eligibility traces 
    elV = lambd * elV + x
    
    # Updated eligibility trace from the 2013 paper
    #    elV = min(1,elV)
    
    # Critic's parameter vector
    v = v + alphaV * delta[i] * elV
    
    elU = lambd * elU + np.multiply((a - agentMean[i]),x)
    wU = wU + alphaW * delta[i] * elU
    
    elS = lambd * elS + np.multiply(((np.power((a - agentMean[i]),2) / np.power(agentStd[i],2)) - 1),x)
    wS = wS + alphaW * delta[i] * elS
    
#     print 'Step: ' + str(i)
#     print 'Joint Angle: ' + str(jointAngle[i]) + ' rads'
#     print 'Target Angle: ' + str(targetElbow[i]) + ' rads'
#     print 'Agent Mean: ' + str(agentMean[i])
#     print 'Agent Std: ' + str(agentStd[i])
#     print 'Action: ' + str(conAngIntToangVel(a)) + ' rad/s, move ' + str(conAngIntToangVel(a)*0.005) + ' rads'
#     print 'New Joint Angle: ' + str(s[0]) + ' rads'
#     print 'Reward: ' + str(r)
#     print 'TD Error: ' + str(delta[i])
#     print '\n'

In [ ]:
# Visualize the learning    
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(t, targetElbow)
ax1.set_ylim(-4,4)
ax1.set_title('Elbow: Target Angle')

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(t, jointAngle)
ax1.set_title('Learned Joint Angle')

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(t, delta)
ax1.set_title('TD Error')

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(t, agentMean)
ax1.set_title('Agent Mean')

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.plot(t, agentStd)
ax1.set_title('Agent Standard Deviation')

##### 